## Introduction to Google Earth Engine

Google Earth Engine is a cloud-based platform for geospatial analysis, enabling users to process large-scale datasets using Google's infrastructure. It offers multiple interfaces for interaction:

- [Explorer](https://explorer.earthengine.google.com/)
- [Code Editor](https://code.earthengine.google.com/)
- [JavaScript API](https://github.com/google/earthengine-api/tree/master/javascript)
- [Python API](https://github.com/google/earthengine-api/tree/master/python)
### Open this Tutorial in Google Colab
<a target="_blank" href="https://colab.research.google.com/github/amirhszd/MachineLearning4RemoteSensing/blob/main/gee/gee_tutorial.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


## Installing Google Earth Engine

Use the following command to install Google Earth Engine (GEE) in your environment. It may already be pre-installed.

In [ ]:
!pip install earthengine-api

#### Authenticating to Earth Engine

To access Google Earth Engine (GEE), first sign up at [GEE Signup](https://signup.earthengine.google.com/).

Authenticate yourself with Google Cloud to gain access to storage and other resources. Run the following code, which will open an authentication page in your browser.

If this is your first time using GEE, you’ll need to create a new project. In the final step, you may also need to register for GEE.

- Select **Unpaid** → **Organization**
- After completing the process, click **Generate Token**
- Copy and paste the token into the output field below.

![authentication](aut.png)

In [ ]:
!earthengine authenticate

#### Importing and Initializing Earth Engine

Once authentication is complete, you can import the `ee` module and initialize Google Earth Engine in your environment. This step establishes a connection to the Earth Engine servers, allowing you to run geospatial computations.

Use the following code to import and initialize Earth Engine:

In [ ]:
import ee
ee.Initialize()

## Exploring GEE Assets and Datasets

Now that we have initialized EE, lets explore various assets in Google Earth Engine (GEE) to familiarize ourselves with the different types of datasets available.

You can browse GEE’s entire data catalog here:
[**GEE Data Catalog**](https://developers.google.com/earth-engine/datasets)

<img src="gee_dc.png" alt="GEE Data Catalog" width="400">


#### Common GEE Datasets

Some of the key datasets available in Google Earth Engine (GEE) include:

- **SRTM Digital Elevation Data**
  [View Dataset](https://developers.google.com/earth-engine/datasets/catalog/CGIAR_SRTM90_V4)

- **ESA World Cover Map**
  [View Dataset](https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v200)

- **Sentinel-2 Multispectral Data**
  [View Dataset](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_HARMONIZED)

- **Landsat-9**
  [View Dataset](https://developers.google.com/earth-engine/datasets/catalog/landsat-9)

<img src="landsat.png" alt="Landsat Data" width="400">

When selecting **Landsat 9 Collection 2** data, you can view details such as availability, data provider, and a sample snippet.

Further down the page, you’ll find:
- **Description** of the dataset
- **Bands** and how they can be accessed
- **Properties** (metadata)
- **Terms of Use**

At the bottom, sample code is available for accessing the data using **JavaScript** and **Python** (if applicable).

## ImageCollection Object

An **ImageCollection** is a time series or stack of images. You can load it using an Earth Engine collection ID or create one using:

- `ee.ImageCollection()`
- `ee.ImageCollection.fromImages()` (from a list of images)

You can also merge existing collections to create new ones.

#### Getting Information from an ImageCollection
ImageCollections information can be printed to the console, but the output is limited to 5,000 elements. Below is an example of retrieving ImageCollection details programmatically.

In [ ]:
# Load the Landsat 9 Collection 2 TOA dataset
collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA')

# Get the total number of images in the collection
total_count = collection.size().getInfo()
print(f"Total images in collection: {total_count}")

# Filter the collection for a specific date range
filtered_collection = collection.filterDate('2022-03-01', '2022-08-01')

# Get the number of images in the filtered collection
filtered_count = filtered_collection.size().getInfo()
print(f"Images between 2021-03-01 and 2021-08-01 of the earth: {filtered_count}")

# Filter for a single day to see how many scenes were captured globally
single_day_collection = collection.filterDate('2022-01-01', '2022-01-02')

# Get the count of images for that day
single_day_count = single_day_collection.size().getInfo()
print(f"Images captured on 2022-01-01 of the earth: {single_day_count}")

.getInfo() converts an Earth Engine object (e.g., ee.Number, ee.Image, ee.Feature, ee.Dictionary) into a Python-native object (e.g., int, dict, list). It retrieves the object’s value from the Earth Engine server, making it accessible in Python.

### Viewing Available Metadata
Each image in an ImageCollection contains metadata (properties) that can be accessed. To check the metadata of an image, we can print its properties:


In [ ]:
# Grabbing the first image of the collection
first_image = collection.first()

# Print the metadata (properties) of the image
print(first_image.propertyNames().getInfo())

# Could also do. Works the same way
print(first_image.getInfo())

### Filtering Using Metadata

In addition to filtering by date, we can filter an **ImageCollection** based on specific metadata properties. Earth Engine provides several filtering options, including:

- **Equal to (`ee.Filter.eq`)**
- **Less than (`ee.Filter.lt`)**
- **Greater than (`ee.Filter.gt`)**
- **Less than or equal to (`ee.Filter.lte`)**
- **Greater than or equal to (`ee.Filter.gte`)**

In [ ]:
# Filter images with cloud cover less than 10%
cloud_lt_10 = collection.filter(ee.Filter.lt('CLOUD_COVER', 20))
count = cloud_lt_10.size().getInfo()
print(count)

### Sorting an ImageCollection

In Earth Engine, we can **sort** an ImageCollection based on metadata properties using `.sort()`.

In [ ]:
# Sort the filtered collection by acquisition date (oldest to newest)
sorted_by_date = collection.sort('system:time_start')

# setting ascending=False, to get descending (big to small values)
sorted_by_cloud_desc = collection.sort('CLOUD_COVER', False)

### Filtering Images for a Specific Bounding Box

To extract all images within a specific region, we use a **bounding box** defined by **longitude** and **latitude** in Google Earth Engine (GEE). (NOT LAT, LON)

In GEE, we can create a bounding box using a **polygon**. A polygon is a vector geometry that defines an area of interest. We will talk more about other types of vector data in a bit.

#### **Creating a Bounding Box**
We define a polygon using the `ee.Geometry.Polygon()` function by specifying corner coordinates **(longitude, latitude)** and `.filterBounds(region)` command

In [ ]:
# Define a bounding box (longitude, latitude)
region = ee.Geometry.Polygon([
    [[-120.0, 35.0],  # Bottom-left
     [-120.0, 37.0],  # Top-left
     [-118.0, 37.0],  # Top-right
     [-118.0, 35.0]]  # Bottom-right
])

# Filter the Landsat 9 collection for this region
collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
    .filterBounds(region)

### Viewing a Sample Image Using Geemap
Now lets pick a range date and a speicifc bounds and use **Geemap**, a Python package for interactive visualization of Earth Engine datasets, to display a sample image.

In [ ]:
!pip install geemap
import geemap

In [ ]:
my_map = geemap.Map()
image = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA').filterDate('2022-03-01', '2022-04-03').filterBounds(ee.Geometry.Polygon([
    [[-120.0, 35.0],  # Bottom-left
     [-120.0, 37.0],  # Top-left
     [-118.0, 37.0],  # Top-right
     [-118.0, 35.0]]  # Bottom-right
])).first()
my_map.addLayer(image, {"bands": ["B4", "B3", "B2"]}, "Landsat 9")
my_map

### Iterating Through an ImageCollection in Google Earth Engine

`ImageCollection` does not support direct iteration in Python. To process images one by one, we convert the `ImageCollection` into a list using `.toList()`, which allows access to individual images sequentially.

#### Steps to Iterate Through an ImageCollection:
1. **Filter the ImageCollection**: Use `.filterDate()` and `.filterBounds()` to define a specific subset of images.
2. **Convert to a List**: Use `.toList(image_collection.size())` to create a list of images.
3. **Iterate Over the List**: Use a loop to extract each image, get its date, and optionally visualize it.

In [ ]:
collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filterDate('2022-03-01', '2022-04-03') \
    .filterBounds(ee.Geometry.Polygon([
        [[-120.0, 35.0],  # Bottom-left
         [-120.0, 37.0],  # Top-left
         [-118.0, 37.0],  # Top-right
         [-118.0, 35.0]]  # Bottom-right
    ]))

# Converting the collection to an ee.List (not a Python list) and determining its size
# You can't have it as a python list because python list is local and EE is on server.
image_list = collection.toList(collection.size())

for index in range(image_list.size().getInfo()):  # Get size in Python
    image = ee.Image(image_list.get(index))  # Extract image from list
    cc = image.get("CLOUD_COVER").getInfo()  # Get cloud cover property
    print(cc)  # Print cloud cover

## Image Object

An `ee.Image` represents a single image in Google Earth Engine, which contains multiple bands. It allows us to perform various operations such as:

- **Selecting Bands: `.select()`** – Extract specific bands from an image.
- **Viewing Projection Information: `.projection()`** – Check an image’s coordinate reference system and projection details.
- **Reprojecting & Rescaling: `.reproject()` and `.reduceResolution()`** – Convert to different coordinate systems and change the scale (ground sampling distance).
- **Performing Operations Between Bands: `.expression()`** – Compute band-based mathematical operations.
- **Computing Normalized Difference: `.normalizedDifference()`** – Calculate NDVI or other indices.
- **Applying Kernels: `.convolve()` with `ee.Kernel`** – Perform edge detection or smoothing.
- **Exporting to NumPy: `geemap.export_to_numpy()`** – Convert image data to a NumPy array (limited pixels).
- **Downloading Images: `geemap.download_ee_image()`** – Download full images for local processing.

---

#### **Retrieving the Projection of an Image in Google Earth Engine**

**What is a Projection?**
A **projection** defines how spatial data is represented on a flat surface. It includes:
- **Coordinate Reference System (CRS)** → Specifies the spatial reference (e.g., EPSG:4326 for WGS84).
- **Affine Transform** → Describes how pixel coordinates relate to geographic coordinates.
- **Scale (Ground Sampling Distance - GSD)** → Defines the pixel resolution in meters.


Understanding the Output
	•	CRS (EPSG:32628) → The spatial reference system (UTM Zone 28N, WGS84).
	•	Transform: [30, 0, 340185, 0, -30, 8808615]
	•	30 → Pixel width (X resolution in meters)
	•	0 → Shear in the X direction (0 means no skew)
	•	340185 → Upper-left corner X coordinate (Easting)
	•	0 → Shear in the Y direction
	•	-30 → Pixel height (negative means north-up)
	•	8808615 → Upper-left corner Y coordinate (Northing)

In [ ]:
### SELECTING A BAND AND GETTING PROJECTION
# selecting a band and getting the projection
image = ee.Image('LANDSAT/LC09/C02/T1_TOA/LC09_228010_20230328')

# Select the Red band (B4)
red_band = image.select('B4')

# Get and print the projection information
projection = red_band.projection()
print(projection.getInfo())

# the output represents the CRS (coordiante Reference system), transform which includes
# x_gds (pixel width), pixel shear in x_direction, upper left corner easting, y_gsd (pixel height), upper left corner northing. All in meters

### getting the native rsolution (scale) in meters
#You can also get the scale of an image as a property using
scale = image.projection().nominalScale()
print(scale)

#### **Reprojection and Rescaling in Google Earth Engine**

Reprojection and rescaling using **Nearest Neighbor interpolation** can be done directly with the `.reproject()` function.

If a **different resampling method** is required, it must be specified using `.resample()` **before** applying `.reproject()`.

✅ .reproject() alone defaults to Nearest Neighbor resampling.
✅ To use a different method, apply .resample() before .reproject().
✅ Supported resampling methods: "nearest" (default), "bilinear", "bicubic".

In [ ]:
# grabbing an image
image = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA').filterDate('2022-03-01', '2022-04-03').filterBounds(ee.Geometry.Polygon([
    [[-120.0, 35.0],  # Bottom-left
     [-120.0, 37.0],  # Top-left
     [-118.0, 37.0],  # Top-right
     [-118.0, 35.0]]  # Bottom-right
])).first()

### REPROJECTION + RESAMPLING using Nearest Neighbour
# use Reproject to change CRS and scale USING NEAREST
image_wgs = image.reproject(crs='EPSG:4326', scale=100)

### REPROJECTION + RESAMPLING using "bilinear" or "bicubic"
# if you want to resample to a specific method use resample before reproject
image = image.resample("bicubic")
image_wgs_bicubic = image.reproject(crs='EPSG:4326', scale=100)

# Compute the difference image
diff_image = image_wgs.select("B4").subtract(image_wgs_bicubic.select("B4"))

# Compute mean difference over the entire image using reduce
mean_difference = diff_image.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=image.geometry())

# Print the mean difference
print(mean_difference.getInfo())

#### **Using Expression to Calculate NDVI in Google Earth Engine**

**Using `.expression()` for Band Math**
In Google Earth Engine (GEE), **`.expression()`** allows defining formulas directly on image bands, similar to **Band Math in ENVI**. Make sure to rename the created image after.

**What is NDVI?**
**Normalized Difference Vegetation Index (NDVI)** is a commonly used index for analyzing vegetation health. It is calculated using the **Near-Infrared (NIR) and Red bands** of satellite imagery:

\[
NDVI = \frac{(NIR - RED)}{(NIR + RED)}
\]

- **NIR (Near-Infrared, Band 5 in Landsat 9)** → Strongly reflected by healthy vegetation.
- **RED (Red, Band 4 in Landsat 9)** → Absorbed by chlorophyll in vegetation.

A higher NDVI value indicates **healthier vegetation**, while lower values suggest **water, bare soil, or unhealthy vegetation**.

In [ ]:
# Load a Landsat 9 image
image = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA').filterDate('2022-03-01', '2022-04-03').filterBounds(ee.Geometry.Polygon([
    [[-120.0, 35.0],  # Bottom-left
     [-120.0, 37.0],  # Top-left
     [-118.0, 37.0],  # Top-right
     [-118.0, 35.0]]  # Bottom-right
])).first()

# Compute NDVI using the standard formula: (NIR - RED) / (NIR + RED)
ndvi = image.expression(
    '(NIR - RED) / (NIR + RED)',
    {
        'NIR': image.select('B5'),  # Near-Infrared Band
        'RED': image.select('B4')   # Red Band
    }
)

ndvi = ndvi.rename('NDVI')  # Rename the output band to 'NDVI'

# Create an interactive map
my_map = geemap.Map()

# Add the NDVI layer to the map
my_map.addLayer(ndvi, {"bands": ["NDVI"]}, "NDVI")

# display map
my_map

#### **Downloading Data in Google Earth Engine Using Geemap**

Data can be downloaded using **`geemap.ee_to_numpy()`** or **`geemap.download_ee_image()`**, each with its own advantages and limitations.

**1. `ee_to_numpy()`**
- Converts the image to a **NumPy array**.
- **Faster**, but has a **pixel limit**, making it unsuitable for large datasets.

**2. `download_ee_image()`**
- Downloads the image **directly to the computer**.
- Can handle **larger images**, but **slower** due to file transfer.
- Requires a **separate library (e.g., `rasterio`)** to load the image after downloading, adding extra processing time.

You could also download an etnire image colleciton if you want using ee_export_image_collection().

In [ ]:
import geemap
# Load a Landsat 9 image within a specific date range and geographic area
roi = ee.Geometry.Polygon([
        [[-120.0, 35.0],  # Bottom-left
         [-120.0, 35.1],  # Top-left
         [-119.9, 35.1],  # Top-right
         [-119.9, 35.0]]  # Bottom-right
    ])
landsat_image = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filterDate('2022-03-01', '2022-04-03') \
    .filterBounds(roi).first().select(["B4"])

# Export image as a NumPy array
array = geemap.ee_to_numpy(image, region=roi, bands = ["B4"])
print(array.shape)


## **Reducers**

Now that we have covered **image collections** and **individual images**, let's move on to **reducers**.

**What Are Reducers?**
Reducers in Google Earth Engine are used to **aggregate data** across **time, space, bands, arrays, and other data structures**. They allow us to summarize large datasets into meaningful statistics.

**Using the `ee.Reducer` Class**
The `ee.Reducer` class defines how data is aggregated. Reducers can compute **simple statistical summaries** such as:
- **Minimum, Maximum, Mean, Median, Standard Deviation**

Or **more complex summaries**, including:
- **Histograms, Linear Regression, Lists of values**

Reducers can be applied in different ways depending on what data needs to be aggregated:

| **Reduction Type** | **Method Used** | **Description** |
|--------------------|----------------|-----------------|
| **Over Time** | `imageCollection.reduce()` | Aggregates images over a time series (e.g., mean of NDVI over time). |
| **Over Space** | `image.reduceRegion()` | Aggregates values over a specific region (e.g., mean temperature in a polygon). |
| **Neighborhoods** | `image.reduceNeighborhood()` | Aggregates values within a moving window (e.g., smoothing with local mean). |
| **Across Bands** | `image.reduce()` | Aggregates multiple spectral bands within a single image (e.g., mean of selected bands). |


#### **Statistical Reducers in Google Earth Engine**

Statistical reducers in Google Earth Engine allow you to compute **summary statistics** over images, image collections, or regions. These reducers can be **combined** using the `.combine()` function, enabling multiple statistical calculations in a single operation.

**Common Statistical Reducers**
Google Earth Engine provides several built-in reducers, including:

- **`ee.Reducer.mean()`** → Computes the mean (average) value.
- **`ee.Reducer.median()`** → Computes the median value.
- **`ee.Reducer.min()`** → Finds the minimum value.
- **`ee.Reducer.max()`** → Finds the maximum value.
- **`ee.Reducer.stdDev()`** → Computes the standard deviation.
- **`ee.Reducer.sum()`** → Computes the sum of values.
- **`ee.Reducer.count()`** → Counts the number of non-null values.
- **`ee.Reducer.histogram()`** → Creates a histogram of values.
- **`ee.Reducer.percentile([percentiles])`** → Computes specific percentiles.

Multiple reducers can be combined using **`.combine()`**.

In [ ]:
from pprint import pprint

# Load a Landsat 9 image within a specific date range and geographic area
landsat_image = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filterDate('2022-03-01', '2022-04-03') \
    .filterBounds(ee.Geometry.Polygon([
        [[-120.0, 35.0],  # Bottom-left
         [-120.0, 37.0],  # Top-left
         [-118.0, 37.0],  # Top-right
         [-118.0, 35.0]]  # Bottom-right
    ])).first()

# Define a reducer that calculates both mean and standard deviation
combined_reducer = ee.Reducer.mean().combine(
    reducer2=ee.Reducer.stdDev(),
    sharedInputs=True
)

# Apply the reducer to compute statistical summaries for each band
image_statistics = landsat_image.reduceRegion(
    reducer=combined_reducer,
    bestEffort=True,
)

# Print the computed mean and standard deviation values
pprint(image_statistics.getInfo())

#### **Reducing an ImageCollection Using Reducers**

To compute statistics over a time series of images in an **ImageCollection**, use **`.reduce()`**, which collapses the collection into a single image by applying a specified reducer **pixel-wise**. Each pixel in the output represents the computed statistic across all images at that location.

**Example Usage**
The following code demonstrates how to:
- Compute the **median** of an image collection.
- Use different reducers (**mean, sum, variance, percentiles**).
- Apply shortcut methods for basic statistics (**min, max, mean**).

![reduce](Reduce_ImageCollection.png)

In [ ]:
# Load an ImageCollection (example: Landsat 9)
imageCollection = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filterDate('2022-03-01', '2022-04-03') \
    .filterBounds(ee.Geometry.Polygon([
        [[-120.0, 35.0], [-120.0, 37.0], [-118.0, 37.0], [-118.0, 35.0]]
    ]))

# Reduce the collection to a single image using different reducers
median_image = imageCollection.reduce(ee.Reducer.median())  # Compute median
mean_image = imageCollection.reduce(ee.Reducer.mean())  # Compute mean
sum_image = imageCollection.reduce(ee.Reducer.sum())  # Compute sum
variance_image = imageCollection.reduce(ee.Reducer.variance())  # Compute variance
percentile_image = imageCollection.reduce(ee.Reducer.percentile([25, 75]))  # Compute percentiles


#### **Combining Reducers to Extract Band Values at a Single Point Over Time**

To extract **band values** at a specific point for each image in an **ImageCollection**, we:
1. **Apply reducers** (e.g., mean, standard deviation) on a selected band.
2. **Use the `.map()` function** to process each image in the collection.
3. **Aggregate the results** into an array to retrieve the extracted values over time.

This approach allows for efficient **time-series extraction** of multiple statistics at a specific location.


In [ ]:
# Define the point of interest (latitude, longitude)
point = ee.Geometry.Point([-119.0, 36.0])

# Load a Landsat 9 ImageCollection within a date range
collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filterDate('2022-01-01', '2022-12-31') \
    .filterBounds(point)

# Function to extract the band value at the point
def extract_band_value(image):
    band_value = image.select('B5').reduceRegion(
        reducer=ee.Reducer.first(),  # Gets the pixel value at the point
        geometry=point,
        scale=30
    )

    # Add the extracted band value and date as properties
    return image.set('date', image.date().format()).set('B5_Value', band_value.get('B5'))

# Apply the function to each image in the collection using the map function
collection_with_values = collection.map(extract_band_value)

# Extract the time series data by aggregate data over the
band_values = collection_with_values.aggregate_array('B5_Value').getInfo()
dates_list = collection_with_values.aggregate_array('date').getInfo()

# Print the time series values
print("B5 Time Series:")
for date, value in zip(dates_list, band_values):
    print(f"{date}: {value}")

## **Geometries and Features**

Google Earth Engine supports various geometry types for spatial analysis:

- **Point** → A single coordinate in a specified projection.
- **LineString** → A sequence of connected points forming a line.
- **LinearRing** → A closed **LineString**, where the start and end points are the same.
- **Polygon** → A list of **LinearRings**, where:
  - The **first ring** defines the **outer boundary (shell)**.
  - Subsequent rings define **inner holes**.
    -
These geometry types enable **spatial operations** such as buffering, intersections, and spatial filtering in Earth Engine.

In [ ]:
# Define a Point geometry (longitude, latitude)
point = ee.Geometry.Point([1.5, 1.5])  # Single coordinate location

# Define a LineString (a sequence of connected points forming a line)
lineString = ee.Geometry.LineString([
    [-35, -10],  # Start point (longitude, latitude)
    [35, -10],   # Second point
    [35, 10],    # Third point
    [-35, 10]    # End point
])

# Define a LinearRing (a closed LineString where the start and end points are the same)
linearRing = ee.Geometry.LinearRing([
    [-35, -10],  # Start point
    [35, -10],   # Second point
    [35, 10],    # Third point
    [-35, 10],   # Fourth point
    [-35, -10]   # Closing the ring (same as the start point)
])

# Define a Rectangle (bounding box using min long, min lat, max long, max lat)
rectangle = ee.Geometry.Rectangle([-40, -20, 40, 20])  # Defines a rectangular bounding box

# Define a Polygon (a closed shape with multiple points)
polygon = ee.Geometry.Polygon(  [
    [ [100.0, 0.0],
      [103.0, 0.0],
      [103.0, 3.0],
      [100.0, 3.0],
      [100.0, 0.0]
    ],
    [
      [101.0, 1.0],
      [102.0, 2.0],
      [102.0, 1.0]
    ]
  ])

# Print the geometries to verify
print("Point:", point.getInfo())
print("LineString:", lineString.getInfo())
print("LinearRing:", linearRing.getInfo())
print("Rectangle:", rectangle.getInfo())
print("Polygon:", polygon.getInfo())

#### **Extracting Information from Geometries in Google Earth Engine**

Once a geometry is defined in Earth Engine, various properties can be retrieved, such as:

- **Area (`.area()`)** → Computes the area of a polygon in square meters.
- **Perimeter (`.length()`)** → Computes the perimeter of a polygon or the length of a LineString.
- **Convert to GeoJSON (`.toGeoJSON()`)** → Exports the geometry in GeoJSON format.
- **Print Coordinates (`.coordinates()`)** → Retrieves the list of coordinates that define the geometry.

In [ ]:
print('Polygon printout: ', polygon.getInfo())

# Print polygon area in square kilometers.
print('Polygon area: ', polygon.area().divide(1000 * 1000).getInfo())

# Print polygon perimeter length in kilometers.
print('Polygon perimeter: ', polygon.perimeter().divide(1000).getInfo())

# Print the coordinates as lists
print('Polygon coordinates: ', polygon.coordinates().getInfo())

#### **Geometric Operations in Google Earth Engine**

Google Earth Engine allows performing various **geometric operations** on features, such as:

- **Buffering (`.buffer()`)** → Expands a point, line, or polygon by a specified distance.
- **Intersection (`.intersection()`)** → Computes the overlapping area between two geometries.
- **Union (`.union()`)** → Merges two geometries into a single shape.
- **Difference (`.difference()`)** → Subtracts one geometry from another.


In [ ]:
import geemap
# Create a map
my_map = geemap.Map()

# Define two circular geometries (buffers around points)
poly1 = ee.Geometry.Point([-50, 30]).buffer(1e6)  # 1 million meter buffer
poly2 = ee.Geometry.Point([-40, 30]).buffer(1e6)  # 1 million meter buffer

# Perform geometric operations
buffer1 = poly1  # Original buffer
buffer2 = poly2  # Original buffer

# Intersection of two geometries
intersection = poly1.intersection(poly2)

# Union (merge both geometries)
union = poly1.union(poly2)

# Difference (part of poly1 that is not in poly2)
difference = poly1.difference(poly2)

# Add geometries to the map
my_map.addLayer(buffer1, {'color': 'red'}, "Buffer 1")
my_map.addLayer(buffer2, {'color': 'blue'}, "Buffer 2")
my_map.addLayer(intersection, {'color': 'green'}, "Intersection")
my_map.addLayer(union, {'color': 'purple'}, "Union")
my_map.addLayer(difference, {'color': 'orange'}, "Difference")

# Display the map
my_map

#### **Converting a Geometry to a Feature in Google Earth Engine**

In Google Earth Engine, you can **convert a geometry into a feature** by adding **attributes (properties)** to it. This means the geometry is no longer just spatial data—it now contains **additional information**.

**Key Concept**
- A **feature** is essentially a **dictionary** where the **geometry** is just one part of it.
- Features allow **storing metadata** such as species names, land cover types, or other relevant properties alongside the geometry.


In [ ]:
from pprint import pprint

# Create a feature with a Point geometry and assign properties
feature = ee.Feature(ee.Geometry.Point([-120, 35])) \
    .set('genus', 'Sequoia') \
    .set('year identified', 1999)

# Retrieve and print a property from the feature
species = feature.get('genus')
print(species.getInfo())  # Output: 'sempervirens'

# Add a new property to the feature called species
feature = feature.set('species', 1)

# Display the updated feature information
pprint(feature.getInfo())

#### **Creating a FeatureCollection in Google Earth Engine**

In Google Earth Engine, a **FeatureCollection** is a collection of multiple **features**, where each feature consists of **a geometry and associated attributes**. This is useful for **grouping spatial data** and performing analysis on multiple features at once.


In [ ]:
# Make a list of Features.
features = [
  ee.Feature(ee.Geometry.Rectangle(30.01, 59.80, 30.59, 60.15), {'name': 'Voronoi'}),
  ee.Feature(ee.Geometry.Point(-73.96, 40.781), {'name': 'Thiessen'}),
  ee.Feature(ee.Geometry.Point(6.4806, 50.8012), {'name': 'Dirichlet'})
]

# Create a FeatureCollection from the list and print it.
fromList = ee.FeatureCollection(features)
fromList.getInfo()

#### **Visualizing an Entire FeatureCollection in Google Earth Engine**

You can also **visualize an entire FeatureCollection** on a map to display all its features at once.
Below is an example of all the states in the CONUS as features and we are accessing this through the TIGER dataset.

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
vis_params = {
    'color': '000000',
    'colorOpacity': 1,
    'pointSize': 3,
    'pointShape': 'circle',
    'width': 2,
    'lineType': 'solid',
    'fillColorOpacity': 0.66,
}
palette = ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
Map.add_styled_vector(
    states, column="NAME", palette=palette, layer_name="Styled vector", **vis_params
)
Map